导入需要的包

In [1]:
from torch.utils.data import DataLoader # 用于构建数据加载器
from torchvision import transforms      # 用于将数据图片格式化
from torch.utils.tensorboard import SummaryWriter   # 用于展示处理过程

创建数据集

In [2]:
import torchvision

# aphids 蚜虫
# armyWorm 粘虫
# beetle 甲虫
# bollWorm 棉铃虫
# grasshopper 蚱蜢
# mites 螨虫
# sawfly 叶蜂
# stemBorer 茎蚜虫

def remove_non_three_channels(image):
    """移除非三通道的图像"""
    if image.shape[0] != 3:
        return None
    else:
        return image

tranform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Lambda(remove_non_three_channels),
])   # 3 256 256

# 训练数据
root_dir = 'dataset/train'
train_dataset = torchvision.datasets.ImageFolder(root=root_dir, transform=tranform)

# 过滤掉None值
train_dataset = [item for item in train_dataset if item[1] is not None]

# 测试数据
root_dir = 'dataset/test'
test_dataset = torchvision.datasets.ImageFolder(root=root_dir, transform=tranform)

# 过滤掉None值
test_dataset = [item for item in test_dataset if item[1] is not None]


In [3]:
writer = SummaryWriter('logs')

In [4]:
# length长度
train_data_size = len(train_dataset)
test_data_size = len(test_dataset)
print("train: ", train_data_size)
print("test: ", test_data_size)



train:  2400
test:  400


In [6]:
# 利用DataLoader加载数据集
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

# 网络模型
from model import *
xiaozu = Xiaozu()
if torch.cuda.is_available():
    print("调用GPU")
    xiaozu = xiaozu.cuda()
else:
    print("调用CPU")

# 损失函数
loss_fn = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()

# 优化器
learning_rate = 0.01    # 学习速率
optimizer = torch.optim.SGD(xiaozu.parameters(), lr=learning_rate)

# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练轮数
epoch = 5

for i in range(epoch):
    print("--------------第 {} 轮训练开始----------------".format(i + 1))
    
    # 训练步骤开始
    for data in train_dataloader:
        torch.cuda.empty_cache()
        imgs, targets = data
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            targets = targets.cuda()
        outputs = xiaozu(imgs)
        loss = loss_fn(outputs, targets)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1
        print("训练次数：{}， Loss：{}".format(total_train_step, loss.item()))
        # print(targets)
    
    # 测试步骤开始
    total_test_loss = 0
    with torch.no_grad:
        for data in test_dataloader:
            imgs, targets = data
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                targets = targets.cuda()
            outputs = xiaozu(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
    print("整体测试集上的Loss：{}".format(total_test_loss))
        



调用GPU
--------------第 1 轮训练开始----------------


OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB. GPU 0 has a total capacity of 1.95 GiB of which 288.81 MiB is free. Including non-PyTorch memory, this process has 1.66 GiB memory in use. Of the allocated memory 1.57 GiB is allocated by PyTorch, and 41.27 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)